$$
\boldsymbol{x} \in \mathbb{R}^{K \times 1} \quad \text{Portfolio’s active factor exposures (e.g., DTS relative to benchmark)}
$$

$$
\boldsymbol{b}_{\text{raw}} \in \mathbb{R}^{K \times 1} \quad \text{Benchmark’s factor exposures (DTS across sectors)}
$$

$$
\boldsymbol{b} \in \mathbb{R}^{K \times 1} \quad \text{Normalized benchmark exposures (defines credit beta direction)}
$$

$$
\Sigma \in \mathbb{R}^{K \times K} \quad \text{Factor return covariance matrix}
$$

$$
\boldsymbol{x}_{\parallel} \in \mathbb{R}^{K \times 1} \quad \text{Projection of portfolio exposure onto credit beta direction}
$$

$$
\boldsymbol{x}_{\perp} \in \mathbb{R}^{K \times 1} \quad \text{Residual exposures orthogonal to credit beta}
$$

$$
\sigma_{\text{beta}} \quad \text{Volatility contribution from credit beta exposure}
$$

$$
\sigma_{\text{resid}} \quad \text{Volatility contribution from residual sector/style exposure}
$$

$$
\sigma_{\text{xs}} \quad \text{Total expected credit tracking error}
$$

$$
\boldsymbol{b} = \frac{\boldsymbol{b}_{\text{raw}}}{\| \boldsymbol{b}_{\text{raw}} \|}
$$

$$
\boldsymbol{x}_{\parallel} = (\boldsymbol{b}^\top \boldsymbol{x}) \cdot \boldsymbol{b}
$$

$$
\boldsymbol{x}_{\perp} = \boldsymbol{x} - \boldsymbol{x}_{\parallel}
$$

$$
\sigma_{\text{beta}} = \sqrt{ \boldsymbol{x}_{\parallel}^\top \Sigma \boldsymbol{x}_{\parallel} }
$$

$$
\sigma_{\text{resid}} = \sqrt{ \boldsymbol{x}_{\perp}^\top \Sigma \boldsymbol{x}_{\perp} }
$$

$$
\sigma_{\text{xs}} = \sqrt{ \boldsymbol{x}^\top \Sigma \boldsymbol{x} } = \sqrt{ \sigma_{\text{beta}}^2 + \sigma_{\text{resid}}^2 }
$$


In [ ]:
import numpy as np

def decompose_credit_te(x: np.ndarray, b_raw: np.ndarray, Sigma: np.ndarray):
    """
    Decomposes portfolio tracking error into credit beta and residual components.

    Parameters
    ----------
    x : np.ndarray of shape (K, 1)
        Portfolio active factor exposures (e.g., DTS relative to benchmark)
    b_raw : np.ndarray of shape (K, 1)
        Benchmark factor exposures (DTS)
    Sigma : np.ndarray of shape (K, K)
        Factor return covariance matrix

    Returns
    -------
    dict
        {
            'total_te': scalar,
            'beta_te': scalar,
            'residual_te': scalar,
            'beta_fraction': scalar,  # beta_te^2 / total_te^2
            'residual_fraction': scalar  # residual_te^2 / total_te^2
        }
    """
    # Normalize benchmark exposure to get credit beta direction
    b = b_raw / np.linalg.norm(b_raw)

    # Project x onto credit beta direction
    x_parallel = (b.T @ x)[0, 0] * b   # scalar projection * unit vector
    x_residual = x - x_parallel

    # Volatility decomposition
    beta_te = np.sqrt(x_parallel.T @ Sigma @ x_parallel)[0, 0]
    residual_te = np.sqrt(x_residual.T @ Sigma @ x_residual)[0, 0]
    total_te = np.sqrt(x.T @ Sigma @ x)[0, 0]

    return {
        "total_te": total_te,
        "beta_te": beta_te,
        "residual_te": residual_te,
        "beta_fraction": beta_te**2 / total_te**2 if total_te > 0 else 0.0,
        "residual_fraction": residual_te**2 / total_te**2 if total_te > 0 else 0.0,
    }


In [ ]:
import numpy as np

def pca_variance_explained(cov_matrix: np.ndarray):
    """
    Perform PCA on a covariance matrix and return the percent of total variance
    explained by the first principal component (PC1).

    Parameters
    ----------
    cov_matrix : np.ndarray of shape (K, K)
        Covariance matrix of K risk factors

    Returns
    -------
    dict
        {
            'explained_variance_pc1': float,
            'eigenvalues': np.ndarray,  # sorted descending
            'eigenvectors': np.ndarray,  # corresponding eigenvectors
        }
    """
    # Eigen decomposition
    eigenvals, eigenvecs = np.linalg.eigh(cov_matrix)

    # Sort eigenvalues and eigenvectors in descending order
    idx = np.argsort(eigenvals)[::-1]
    eigenvals_sorted = eigenvals[idx]
    eigenvecs_sorted = eigenvecs[:, idx]

    # Total variance = trace of covariance matrix
    total_variance = np.sum(eigenvals_sorted)

    # Variance explained by PC1
    explained_variance_pc1 = eigenvals_sorted[0] / total_variance

    return {
        'explained_variance_pc1': explained_variance_pc1,
        'eigenvalues': eigenvals_sorted,
        'eigenvectors': eigenvecs_sorted
    }